In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = SparkSession \
    .builder \
    .appName("File Streaming Demo") \
    .master("local[3]") \
    .config("spark.streaming.stopGracefullyOnShutdown", "true") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 8)

In [2]:
# df_raw = spark.read.format("text").load("./dataset/examples.txt")

In [3]:
df_raw = spark.readStream.format("socket").option("host","172.20.0.1").option("port", "9999").load()
df_raw.printSchema()

root
 |-- value: string (nullable = true)



In [4]:
df_words = df_raw.withColumn("words", F.split("value", " "))
df_explode = df_words.withColumn("word", F.explode("words")).drop("value", "words")
df_agg = df_explode.groupBy("word").count()

In [5]:
# df_agg.writeStream.format("console").outputMode("update").start().awaitTermination()
stream = (
    df_agg
    .writeStream
    .format("console")
    .outputMode("complete")
    .start()
    )

stream.awaitTermination()

StreamingQueryException: Query [id = fddd8ca1-3876-43de-8dcf-af30521eb0fd, runId = 035e2665-beca-4c23-b57e-91f1732df04c] terminated with exception: Connection refused (Connection refused)